In [1]:
%%capture
#@title set up auto-formatting of cells in notebook

from IPython.display import HTML, display


def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )
get_ipython().events.register("pre_run_cell", set_css)

In [2]:
#@title check  system stats
from psutil import virtual_memory
import os
ram_gb = round(virtual_memory().total / (1024**3), 1)
print(f'Runtime has {ram_gb} gigs of memory and {os.cpu_count()} processors')

Runtime has 12.7 gigs of memory and 2 processors


In [3]:
!nvidia-smi

Fri Dec  9 02:05:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install --upgrade transformers==4.21.3 
!pip install -U sentencepiece -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 12.0 MB/s 
     |████████████████████████████████| 182 kB 28.2 MB/s 
     |████████████████████████████████| 6.6 MB 21.7 MB/s 
     |████████████████████████████████| 1.3 MB 13.1 MB/s 


#* load model

In [5]:
hf_tag = "pszemraj/opt-350m-email-generation" #@param {type:"string"}

In [6]:
import torch
from transformers import pipeline
email_gen = pipeline(
    'text-generation', 
    hf_tag, 
    use_fast=False,
    device = 0 if torch.cuda.is_available() else -1,
)


In [7]:
import pprint as pp

def call_model(prompt:str, 
                num_beams=4, 
                min_length=4, 
                max_length=64,
                no_repeat_ngram_size=3,
                temperature=0.3,
            ):
    """ helper function for the pipeline object """
    result = email_gen(

        prompt, 
        min_length=min_length, max_length=max_length,
        no_repeat_ngram_size = no_repeat_ngram_size,
        repetition_penalty=3.5,
        length_penalty=0.8,
        temperature=temperature,
        remove_invalid_values=True,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
        do_sample=False,
        early_stopping=True,
        num_beams=num_beams,
        return_full_text=False,

    )

    w_prompt = f"<PROMPT>{prompt}<END-OF-PROMPT>"
    response = result[0]['generated_text']
    pp.pprint(w_prompt + response)

# generate responses

- higher `num_beams` = better quality, more compute intensive
- adjust `max_length` for longer responses which are also obviously more compute intensive
 

In [8]:
prompt = """
Hello, 
As the President of the University, 
I am writing to request your help in completing a course completion survey form."""""

call_model(prompt, max_length=128, num_beams=4)

('<PROMPT>\n'
 'Hello, \n'
 'As the President of the University, \n'
 'I am writing to request your help in completing a course completion survey '
 'form.<END-OF-PROMPT>\n'
 'The purpose of this survey is to gather feedback from <NUMBER> current '
 'students on their experience at UofH.\n'
 'Please take a few minutes to complete the survey and return it back to me as '
 'soon as possible.\n'
 'Thank you very much for your time.\n'
 'Sincerely, John Lavorato Professor of Business, University of Houston '
 'College of Engineering, Department of Finance and Business Administration '
 '(UofH MBA) <PHONE> Cell <EMAIL> Fax <')


In [9]:
prompt = """
Hello, 
As the President of the University, 
I am writing to request your help in completing a course completion survey form."""
call_model(prompt, max_length=128, num_beams=4)


('<PROMPT>\n'
 'Hello, \n'
 'As the President of the University, \n'
 'I am writing to request your help in completing a course completion survey '
 'form.<END-OF-PROMPT>\n'
 'The purpose of this survey is to gather feedback from <NUMBER> current '
 'students on their experience at UofH.\n'
 'Please take a few minutes to complete the survey and return it back to me as '
 'soon as possible.\n'
 'Thank you very much for your time.\n'
 'Sincerely, John Lavorato Professor of Business, University of Houston '
 'College of Engineering, Department of Finance and Business Administration '
 '(UofH MBA) <PHONE> Cell <EMAIL> Fax <')


In [10]:
prompt = """
Hello, 
As the President of the University, 
I am writing to request your help in completing a course completion survey form."""
call_model(prompt, max_length=64, num_beams=8)


('<PROMPT>\n'
 'Hello, \n'
 'As the President of the University, \n'
 'I am writing to request your help in completing a course completion survey '
 'form.<END-OF-PROMPT>\n'
 'The purpose of this survey is to gather feedback from <NUMBER> current and '
 'former students on their experiences at UofH.\n'
 'Please take a few minutes to')


In [11]:
prompt = """
Hello, 
As the President of the University, 
I am writing to request your help in completing a course completion survey form."""
call_model(prompt, max_length=128, num_beams=8)


('<PROMPT>\n'
 'Hello, \n'
 'As the President of the University, \n'
 'I am writing to request your help in completing a course completion survey '
 'form.<END-OF-PROMPT>\n'
 'The purpose of this survey is to gather feedback from <NUMBER> current and '
 'former students on their experiences at UofH.\n'
 'Please complete the survey no later than Friday, October 00th by sending it '
 'back to me via fax no.\n'
 '<PHONE>.\n'
 'Thank you very much for your time.\n'
 'Best Regards, _______________________________________________UofH Office of '
 'Admissions <EMAIL> (See attached file: uofh0000_sample.pdf)')
